<a href="https://colab.research.google.com/github/bijjula/aimodeltraining1/blob/main/aimodeltraining1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets pandas transformers scikit-learn #datasets library is used for loading dataset present in hugging face into our collab in cache
                                                        #pandas used for loading data into coulms and used for viewing
                                                        # transformer library used for tokenising the input and adding ids
                                                        #sklearn or scikit learn both are same

In [3]:
pip install --upgrade transformers #upgrading of transformer library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 56.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.2
    Uninstalling transformers-4.53.2:
      Successfully uninstalled transformers-4.53.2


In [4]:
!pip install --upgrade datasets # upgrading dataset provide upgradation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [5]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("RuteshReddyB/etiv2", split="train") #we can take any subset present


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/609 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/5.22k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/257 [00:00<?, ? examples/s]

In [30]:
# Step 1: Get all Telugu labels (list)
telugu_labels = dataset["Telugu"]  # or whatever your column is named
print(telugu_labels, " ", len(telugu_labels))
# Step 2: Fit the label encoder ONCE on the full list
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(telugu_labels)
print(telugu_labels[0])
# Step 3: Map label IDs using transform (not fit_transform!)
def add_label_id(example):
    example["label_id"] = label_encoder.transform([example["Telugu"]])[0]
    #print(label_encoder.transform([example["Telugu"]])[0])
    #print(example["label_id"])
    return example

dataset = dataset.map(add_label_id)
print(dataset)
print(dataset[0])
num_classes = len(label_encoder.classes_)
print("Number of classes (unique Telugu words):", num_classes)
print("Number of samples in the dataset:", len(dataset))

Column(['తండ్రి', 'తల్లి', 'అన్న / తమ్ముడు', 'అక్క / చెల్లి', '\tస్నేహితుడు'])   257
తండ్రి


Map:   0%|          | 0/257 [00:00<?, ? examples/s]

Dataset({
    features: ['English', 'Telugu', 'label_id'],
    num_rows: 257
})
{'English': 'Father', 'Telugu': 'తండ్రి', 'label_id': 91}
Number of classes (unique Telugu words): 215
Number of samples in the dataset: 257


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'Telugu' column from the dataset contains the labels
raw_labels = dataset["Telugu"]

# Remove empty strings, None, whitespace-only strings
clean_labels = [label for label in raw_labels if label and label.strip() != '']

# Get sorted unique class names
unique_labels = sorted(set(clean_labels))

# Refit LabelEncoder
label_encoder = LabelEncoder()  #think like creation of empty list
label_encoder.fit(unique_labels)

# Verify
#print("Cleaned classes:", label_encoder.classes_)
print("First label for class 0:", label_encoder.inverse_transform([2]))

First label for class 0: ['\tస్నేహితుడు']


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # Just to tokenize words
def tokenize(batch):
    tokens = tokenizer(batch["English"], padding="max_length", truncation=True, max_length=8)
    tokens["labels"] = batch["label_id"]
    return tokens

tokenized = dataset.map(tokenize, batched=False)
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/257 [00:00<?, ? examples/s]

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_classes
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

from transformers import Trainer,TrainingArguments,DataCollatorWithPadding

training_args = TrainingArguments(
    output_dir="./telugu_classifier",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_strategy="epoch",
    logging_dir="./logs",
    report_to="none"  # Avoids wandb popup
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,   # tokenized with input_ids, attention_mask, labels
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer)
)

trainer.train()
trainer.save_model("./telugu_classifier")

/tmp/ipython-input-13-2019769337.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


In [ ]:
def translate(word):
    inputs = tokenizer(word, return_tensors="pt", truncation=True, padding=True, max_length=8)
    outputs = model(**inputs)

    print("Logits:", outputs.logits)
    pred_class = torch.argmax(outputs.logits, dim=-1).item()

    print("Predicted class index:", pred_class)
    return str(label_encoder.inverse_transform([pred_class])[0])
translate("Father")


Logits: tensor([[ 0.5435,  0.4815,  0.0464,  0.0274, -0.1732, -0.1208,  0.2243,  0.1308,
         -0.1203,  0.7390,  0.3225,  0.2568,  0.3380, -0.3400, -0.0403,  0.0345,
          0.0324,  0.2160,  0.2982,  0.0456, -0.0761,  0.0722,  0.1675,  0.2742,
         -0.2123,  0.2866,  0.8059, -0.2010,  0.5046, -0.1906, -0.0897,  0.1155,
          0.1027,  0.0864, -0.2624,  0.2684,  0.1912,  0.4464,  0.6043,  0.0886,
         -0.5895, -0.0899,  0.4858, -0.1315,  0.4476,  0.0464,  0.2987,  0.5148,
         -0.2077,  0.2410, -0.1567,  0.5728,  0.3224,  0.4456, -0.2552,  0.1599,
          0.5229,  0.2528, -0.0515, -0.1551, -0.2271,  0.5145,  0.3532,  0.5737,
          0.3480, -0.0057, -0.1872,  0.2136, -0.3415,  0.2717,  0.0962,  0.5936,
          0.1445,  0.1383,  0.0882, -0.0950, -0.7880,  0.0489,  0.0656, -0.1520,
         -0.1147,  0.2250, -0.1192,  0.4315, -0.4596, -0.4850, -0.2852, -0.1872,
         -0.1852, -0.1140,  0.4865, -0.0264, -0.0606,  0.5139, -0.2418, -0.1622,
         -0.0787,  0

'బైక్'

In [ ]:
import torch

def translate(word):
    inputs = tokenizer(word, return_tensors="pt", truncation=True, padding=True, max_length=8)
    outputs = model(**inputs)
    pred_class = torch.argmax(outputs.logits, dim=-1).item()
    return str(label_encoder.inverse_transform([pred_class])[0])

translate("Mo")


'రెండు'